In [6]:
# Import required libraries and functions.
import os
import pandas as pd
import random
from datetime import datetime, timedelta

from utils import (
    get_times, calc_coherence, run_coherence, get_max_corr,
    get_frame_files, get_strain_data, find_max_corr_channel, plot_max_corr_chan, give_group_v2
)
from gwpy.timeseries import TimeSeries

# Set the parameters (instead of using argparse in a notebook, define them directly):
lowfreq = 55      # Lower boundary for frequency in DeepClean
highfreq = 65     # Upper boundary for frequency in DeepClean
savedir = os.path.join(os.getcwd(), "K1_automatic", "1368986010")  # Example savedir
ifo = "K1"
dur = 900         # Duration of data in seconds

print("Parameters:")
print("  lowfreq =", lowfreq)
print("  highfreq =", highfreq)
print("  savedir =", savedir)
print("  ifo =", ifo)
print("  dur =", dur)

Parameters:
  lowfreq = 55
  highfreq = 65
  savedir = /disk/home/shu-wei.yeh/coherence-monitor/K1_automatic/1368986010
  ifo = K1
  dur = 900


In [7]:
# Robustly extract a numeric time from the savedir.
savedir_abs = os.path.abspath(savedir)
time_str = os.path.basename(os.path.normpath(savedir_abs))
if not time_str.isdigit():
    time_str = os.path.basename(os.path.dirname(savedir_abs))
if not time_str.isdigit():
    raise ValueError(f"Could not extract a numeric time from savedir '{savedir}'")
time_val = int(time_str)
print("Current time is:", time_val)

Current time is: 1368986010


In [8]:
# Generate the plot of maximum coherence channels.
# FFT is set to 10 and the duration for plot titles is taken from dur.
vals = plot_max_corr_chan(savedir, fft=10, ifo=ifo, duration=dur, flow=lowfreq, fhigh=highfreq)

Data acquired; generating plots...
Plots saved successfully.


In [9]:
# Display the head of the DataFrame
vals.head()

,frequency,channel1,corr1,channel2,corr2,group1,group2
550,55.0,K1:PEM_SEIS_BS_GND_Y_OUT_DQ_1368986010_1368986...,0.074651,K1:PEM_MIC_OMC_BOOTH_OMC_Z_OUT_DQ_1368986010_1...,0.066296,PEM,PEM
551,55.1,K1:PEM_MIC_OMC_BOOTH_OMC_Z_OUT_DQ_1368986010_1...,0.032529,K1:PEM_SEIS_BS_GND_Z_OUT_DQ_1368986010_1368986...,0.024466,PEM,PEM
552,55.2,K1:PEM_SEIS_BS_GND_Z_OUT_DQ_1368986010_1368986...,0.074881,K1:PEM_SEIS_BS_GND_Y_OUT_DQ_1368986010_1368986...,0.061589,PEM,PEM
553,55.3,K1:PEM_MIC_OMC_TABLE_AS_Z_OUT_DQ_1368986010_13...,0.044395,K1:PEM_MIC_OMC_BOOTH_OMC_Z_OUT_DQ_1368986010_1...,0.041983,PEM,PEM
554,55.4,K1:PEM_SEIS_BS_GND_X_OUT_DQ_1368986010_1368986...,0.109170,K1:PEM_SEIS_BS_GND_Z_OUT_DQ_1368986010_1368986...,0.091231,PEM,PEM


In [22]:
# Select rows based on frequency and coherence thresholds.
vals_selc = vals.loc[
    (vals['frequency'] > lowfreq) &
    (vals['frequency'] < highfreq) &
    ((vals['corr1'] > 0.2) | (vals['corr2'] > 0.2))
]

# Display filtered results.
vals_selc.head()
print(vals_selc)

Empty DataFrame
Columns: [frequency, channel1, corr1, channel2, corr2, group1, group2]
Index: []


In [ ]:
# Obtain unique witness channels from both 'channel1' and 'channel2' columns.
channels1 = vals_selc.sort_values('corr1', ascending=False)\
                      .drop_duplicates(subset=['channel1'])['channel1'].tolist()
channels2 = vals_selc.sort_values('corr2', ascending=False)\
                      .drop_duplicates(subset=['channel2'])['channel2'].tolist()

channels = channels1.copy()
for c in channels2:
    if c not in channels:
        channels.append(c)

print("Unique witness channels:")
print(channels)

In [ ]:
# Write the channel list to a file.
with open('chanlist_O4.ini', 'w') as f:
    # The first line is the strain channel.
    f.write(f"{ifo}:CAL-CS_PROC_DARM_STRAIN_DBL_DQ\n")
    for c in channels:
        # Replace the first underscore with a dash.
        new_c = c.replace('_', '-', 1)
        f.write(new_c + '\n')

print("Channels written to chanlist_O4.ini")

In [ ]:
# Open and read the contents of the INI file to verify that the channels are correctly written.
ini_file = 'chanlist_O4.ini'
with open(ini_file, 'r') as f:
    contents = f.read()
print("Contents of chanlist_O4.ini:")
print(contents)